In [1]:
$LOAD_PATH << File.dirname(__FILE__) + "/../lib"
$LOAD_PATH << File.dirname(__FILE__) + "/../../ruby-audio_stream/lib"

require 'synthesizer'
require 'audio_stream'
require_relative 'buffer_player'

include AudioStream
include Synthesizer

soundinfo = SoundInfo.new(
  channels: 2,
  samplerate: 44100,
  window_size: 1024,
  format: RubyAudio::FORMAT_WAV|RubyAudio::FORMAT_PCM_16
)

synth = PolySynth.new(
  oscillators: [
    Oscillator.new(
      source: OscillatorSource::Sine.instance,
      freq_modulators: [
        PolySynth.new(
          oscillators: [
            Oscillator.new(
              source: OscillatorSource::Sine.instance,
              ratio_freq: 8,
            ),
          ],
          amplifier: Amplifier.new(
            volume: ModulationValue.new(0.0)
              .add(Modulation::Adsr.new(
                attack: 0.0,
                hold: 0.0,
                decay: 0.0,
                sustain: 1.0,
                release: 0.0
              ), depth: 1.0),
            ),
          soundinfo: soundinfo,
        ),
      ],
    ),
  ],
  amplifier: Amplifier.new(
    volume: ModulationValue.new(0.0)
      .add(Modulation::Adsr.new(
        attack: 0.0,
        hold: 0.0,
        decay: 2.0,
        sustain: 0.0,
        release: 0.0
      ), depth: 1.0)
      .add(Modulation::Lfo.new(
        rate: 0.2
      ), depth: 0.2),
    ),
  soundinfo: soundinfo,
)

BufferPlayer.play(soundinfo) {|player|
  synth.note_on(Note.new(72))
  Rate.sec(2).frame(soundinfo).ceil.times {
    player.write(synth.next)
  }
  synth.note_off(Note.new(72))
}
nil

"<audio controls src='output_tmp_1641192830_265592.wav'></audio>"